In [119]:
import requests, re, pandas as pd
import time, os, json
from bs4 import BeautifulSoup

TOTAL_RESTAURANT = 1000
base_url = 'https://www.tripadvisor.com/RestaurantSearch'
headers = {
    'User-Agent':  'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36',
}

REST_LIST_DIR = 'cache/rest_list'
REST_DIR = 'cache/restaurants'
FULL_DATASET_DIR = './restaurant_dataset.csv'


if not os.path.exists(REST_LIST_DIR):
    os.makedirs(REST_LIST_DIR)

for idx in range(0, TOTAL_RESTAURANT, 30):
    if not os.path.exists(REST_LIST_DIR+'/NY_'+str(idx)+'.html'):
        parameter_dictionary = {'Action':'PAGE', 'ajax':'1', 'availSearchEnabled':'false', 'sortOrder':'popularity', 
                                'geo':'60763', 'itags':'10591', 'geobroaden':'false', 'o':'a'+str(idx)}
        response = requests.get(url, parameter_dictionary, headers=headers)
        f = open(REST_LIST_DIR+'/NY_'+str(idx)+'.html', 'w', encoding='utf-8')
        f.write(response.text)
        f.close()
        time.sleep(3)


In [40]:
rest_names = []
rest_urls = []
for idx in range(0, TOTAL_RESTAURANT, 30):
    f = open('cache/rest_list/NY_'+str(idx)+'.html', 'r', encoding='utf-8')
    html = f.read()
    soup = BeautifulSoup(html)
    for rest in soup.body.find_all('a', class_='Lwqic Cj b'):
        rest_names.append(rest.text)
        rest_urls.append("https://www.tripadvisor.com"+str(rest).split('"')[3])
    f.close()

In [42]:
if not os.path.exists(REST_DIR):
    os.makedirs(REST_DIR)
    
for url in rest_urls:
    file_name = url.split('-Reviews-')[1].split('-New_York_City')[0]+'.html'
    if not os.path.exists(REST_DIR+ '/'+ file_name):
        response = requests.get(url, headers=headers)
        f = open(REST_DIR+ '/'+ file_name, 'w', encoding='utf-8')
        f.write(response.text)
        f.close()
        time.sleep(3)

In [121]:
if not os.path.exists(FULL_DATASET_DIR):
    
    url_list = []
    name_list = []
    rating_list = []
    rating_food_list = []
    rating_service_list = []
    rating_value_list = []
    cuisine_sublist_list = []
    review_num_list = []
    neighbor_list = []
    price_level_low_list = []
    price_level_high_list = []
    strong_tag_sublist_list = []
    comments_list = []
    for rest_name in os.listdir(REST_DIR):
        try:

            f = open(REST_DIR + '/' + rest_name, 'r', encoding='utf-8')
            html = f.read()
            soup = BeautifulSoup(html)
            f.close()

            url = str(soup.find_all('link', hreflang="en")[0]).split('"')[1]

            name = soup.find_all('h1', class_='HjBfq')[0].text

            bubble = soup.find_all('svg',class_='UctUV d H0')[0]
            regex = r"(\d+\.\d+) of 5 bubbles"
            match = re.search(regex, str(bubble))
            rating = float(match.group(1))

            bubbles = soup.find_all('span', class_='vzATR')
            regex = r"\d+"
            rating_food = float(re.search(regex, str(bubbles[0].span)).group(0))/10
            rating_service = float(re.search(regex, str(bubbles[1].span)).group(0))/10
            rating_value = float(re.search(regex, str(bubbles[2].span)).group(0))/10

            review_num = int(soup.find_all('span', class_='AfQtZ')[0].text.replace(',', '').split(' ')[0])

            neighbor = soup.find_all('span', class_='yEWoV OkcwQ')[0].div.text

            price_cuisine = soup.find_all('a', class_='dlMOJ')
            price_text = price_cuisine[0].text.replace(' ', '')
            if '-' not in price_text:
                price_level_low = len(price_text)
                price_level_high = len(price_text)
            else:
                splited_price = price_text.split('-')
                price_level_low = len(splited_price[0])
                price_level_high = len(splited_price[1])

            cuisine_sublist = []
            for cuisine in price_cuisine[1:]:
                cuisine_sublist.append(cuisine.text)


            strong_tag_sublist = []
            try:
                for strong_tag in soup.find_all('div', class_='ui_tagcloud_group')[0].find_all('span', class_='ui_tagcloud')[1:]:
                    strong_tag_sublist.append(strong_tag.text[:-1])
            except:
                pass

            comments = ''
            for title in soup.find_all('span', class_='noQuotes'):
                comments = comments + title.text
            for comment in soup.find_all('p', class_='partial_entry'):
                comments = comments + comment.text

            url_list.append(url)
            name_list.append(name)
            rating_list.append(rating)
            rating_food_list.append(rating_food)
            rating_service_list.append(rating_service)
            rating_value_list.append(rating_value)
            cuisine_sublist_list.append(cuisine_sublist)
            review_num_list.append(review_num)
            neighbor_list.append(neighbor)
            price_level_low_list.append(price_level_low)
            price_level_high_list.append(price_level_high)
            strong_tag_sublist_list.append(strong_tag_sublist)
            comments_list.append(comments)

        except:
            print(rest_name+' skipped')
            pass
    
    df_raw = {'Name':name_list, 'url':url_list, 'Rating':rating_list, 'Rating_food':rating_food_list, 'Rating_service':rating_service_list,
          'Rating_value':rating_value_list, 'cuisine':cuisine_sublist_list, 'Review_num':review_num_list, 'Neighborhood':neighbor_list,
          'Price_low':price_level_low_list, 'Price_high':price_level_high_list,
          'Strong_tag':strong_tag_sublist_list, 'Comments':comments_list}
    df = pd.DataFrame(df_raw)
    df.to_csv(FULL_DATASET_DIR)

else:
    print('Restaurant dataset already exists. No need to crawl.')

Restaurant dataset already exists. No need to crawl.


In [69]:
name_list = []
for rest_name in os.listdir(REST_DIR):
    
    f = open(REST_DIR + '/' + rest_name, 'r', encoding='utf-8')
    html = f.read()
    soup = BeautifulSoup(html)
    f.close()
    
    name = soup.find_all('h1', class_='HjBfq')[0].text
    

12_Chairs.html
2 3
15_EAST_Tocqueville.html
4 4
230_Fifth.html
2 3
2nd_Avenue_Deli.html
2 3
2_Bros_Pizza.html
1 1
44_X.html
2 3
53rd_6th_Halal.html
1 1
54_Below.html
2 3
5_Napkin_Burger.html
2 3
99_Cent_Fresh_Pizza.html
1 1
ABA_Turkish_Restaurant.html
2 3
ABC_Cocina.html
2 3
ABC_Kitchen.html
2 3
Adrienne_s_Pizzabar.html
2 3
Ai_Fiori.html
4 4
Ajisen_Ramen.html
1 1
Aldea.html
4 4
Aldo_Sohm_Wine_Bar.html
2 3
Almond.html
2 3
Alta.html
2 3
Amarone_Scarlatto.html
2 3
Amelie.html
2 3
Amma.html
2 3
Amsterdam_Ale_House.html
2 3
Amy_Ruth_s_Home_Style_Southern_Cuisine.html
2 3
Andrews_Coffee_Shop.html
1 1
Anejo.html
2 3
Angelo_s_Pizza.html
2 3
Antique_Garage.html
2 3
Antoya_BBQ.html
2 3
An_nam.html
2 3
Apotheke.html
2 3
Applejack_Diner.html
2 3
Aquavit.html
4 4
Aretsky_s_Patroon.html
4 4
Ariana_Afghan_Kabab_Restaurant.html
2 3
Armani_Ristorante.html
4 4
Arno_Ristorante.html
2 3
Arriba_Arriba_Mexican_Restaurant.html
2 3
Artichoke_Basille_s_Pizza.html
1 1
Arturo_s.html
2 3
Astro_Restaurant.html
1 1

Gallaghers_Steakhouse.html
4 4
Gaonnuri.html
4 4
Gato.html
4 4
Gelso_Grand.html
2 3
Gemma.html
2 3
George_s_Restaurant.html
2 3
Gigino_Trattoria.html
2 3
Gigino_Wagner_Park.html
2 3
Giorgio_s_of_Gramercy.html
2 3
Glass_House_Tavern.html
2 3
GMT_Tavern.html
2 3
Gnocco.html
2 3
Good_Enough_to_Eat.html
2 3
Gotham_Pizza.html
1 1
Gotham_Restaurant.html
4 4
Gramercy_Tavern.html
4 4
Grand_Bar_and_Salon.html
2 3
Grand_Central_Oyster_Bar_Inc.html
2 3
Greek_Kitchen.html
2 3
Grimaldi_s.html
2 3
Guantanamera.html
2 3
Gyu_Kaku_Japanese_BBQ.html
2 3
Hakata_Ippudo_NY.html
2 3
Hama_Japanese_Cuisine.html
2 3
Hangawi.html
4 4
Harlem_Tavern.html
2 3
Harney_Sons_Soho.html
2 3
Harry_Cipriani.html
4 4
Harry_s.html
4 4
Harry_s_Italian_Pizza_Bar.html
2 3
Hatsuhana_Sushi_Restaurant.html
2 3
Havana_Central_Restaurant_Rum_Bar.html
2 3
Havana_NY.html
2 3
Haven_Rooftop.html
2 3
Hearth.html
4 4
Heidelberg_Restaurant.html
2 3
Hell_s_Kitchen.html
2 3
Hendriks.html
2 3
Her_Name_Is_Han.html
2 3
Hide_Chan_Ramen_Midtown_

Pera_Soho.html
2 3
Pergola_Des_Artistes.html
2 3
Periyali.html
2 3
Perrine.html
4 4
Perry_St.html
4 4
Pershing_Square_Restaurant.html
2 3
Per_Se.html
4 4
Pete_s_Tavern.html
2 3
Petite_Boucherie.html
2 3
Petit_Poulet.html
2 3
Piacere.html
2 3
Piccola_Cucina_Estiatorio.html
2 3
Piccola_Cucina_Osteria.html
2 3
Pig_and_Khao.html
2 3
Ping_s.html
2 3
Pio_Pio_6.html
2 3
Pio_Pio_8.html
2 3
Pisticci.html
2 3
PizzArte.html
2 3
Pi_Greek_Bakerie.html
2 3
PJ_Carney_s.html
2 3
Playwright_Celtic_Pub.html
2 3
Playwright_Irish_Pub.html
2 3
Please_Don_t_Tell.html
2 3
Pokeworks.html
2 3
Pomodoro_Rosso.html
2 3
Ponche_Taqueria_Cantina.html
1 1
Pongsri_Thai_Restaurant.html
2 3
Porteno.html
2 3
Porter_House.html
4 4
Premier_Cafe.html
1 1
Pret_A_Manger.html
1 1
Prince_Street_Pizza.html
1 1
PRINT.html
4 4
Prune.html
2 3
Puglia_Restaurant.html
2 3
Pure_Fresh.html
1 1
Pure_Thai_Cookhouse.html
2 3
Pylos.html
2 3
P_J_Clarke_s.html
2 3
P_J_Clarke_s_Lincoln_Square.html
2 3
P_J_Clarke_s_On_The_Hudson.html
2 3
P_S_Ki

In [97]:
f = open('cache/restaurants/Valhalla.html', 'r', encoding='utf-8')
html = f.read()
soup = BeautifulSoup(html)
f.close()

In [112]:
soup.find_all('h1', class_='HjBfq')[0].text

'Zuma'